In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# Login credentials and instantiation of AnimalShelter class

username = "aacuser"
password = "greatpassword"
shelter = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    # Add the logo with link to Client's homepage (www.snhu.edu)
    html.A(
        html.Img(src='/assets/Grazioso Salvare Logo.png', style={'height': '300px', 'width': 'auto'}),
        href="http://www.snhu.edu", target="_blank"
    ),
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Douglas Rowland Dashboard '))),
    html.Hr(),
    
    # Add filter options using radio buttons
    dcc.RadioItems(
        id='rescue-type-filter',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Reset Filters', 'value': 'reset'}
        ],
        value='reset',  # Default selection
        labelStyle={'display': 'inline-block', 'margin-right': '10px'},
        
        # Border for the radio buttons for aesthetics
        style={
        'border': '2px solid #ccc',  # Light grey border
        'padding': '10px',           # Padding around the radio buttons
        'border-radius': '5px',      # Rounded corners
        'margin-bottom': '20px',     # Spacing below the radio buttons
        'width': '50%',              # Adjust width if needed
        'background-color': '#f9f9f9' # Light background color
    }
    ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        
        style_table={'overflowX': 'auto'},
        page_size=10,  # Limit to 10 rows per page
        sort_action='native',  # Enable sorting
        filter_action='native',  # Enable filtering
        row_selectable='single',  # Allow single row selection for the mapping callback
        selected_rows=[0],  # Select the first row by default

    ),
    html.Br(),
    html.Hr(),
    html.Div([
        # Pie chart placeholder
        dcc.Graph(id='pie-chart-id', style={'width': '50%', 'display': 'inline-block'}),
        
        # Map placeholder
        html.Div(id='map-id', style={'width': '50%', 'display': 'inline-block'}),
    ], style={'display': 'flex', 'flex-direction': 'row'})  # Flexbox layout for side-by-side display
])


#############################################
# Interaction Between Components / Controller
#############################################
# Callback to update the data table based on filter selection
@app.callback(
    Output('datatable-id', 'data'),
    [Input('rescue-type-filter', 'value')]
)
def update_table(rescue_type):
    # Default query returns all records
    query = {}
    
    # Apply filtering based on the rescue type
    if rescue_type == 'water':
        query = {
            "breed": {"$in": ["Labrador Retriever Mix", 
                              "Chesapeake Bay Retriever", 
                              "Newfoundland"
                             ]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif rescue_type == 'mountain':
        query = {
            "breed": {"$in": ["German Shepherd", 
                              "Alaskan Malamute", 
                              "Old English Sheepdog", 
                              "Siberian Husky", 
                              "Rottweiler"
                             ]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif rescue_type == 'disaster':
        query = {
            "breed": {"$in": ["Doberman Pinscher", 
                              "German Shepherd", 
                              "Golden Retriever", 
                              "Bloodhound", 
                              "Rottweiler"
                             ]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }

    # Fetch filtered data from MongoDB using the CRUD module
    filtered_data = shelter.read(query)
    
    # Convert the filtered data to a pandas DataFrame and remove '_id'
    df_filtered = pd.DataFrame.from_records(filtered_data)
    df_filtered.drop(columns=['_id'], inplace=True)
    
    # Return the filtered data to update the DataTable
    return df_filtered.to_dict('records')

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):

    # Check if viewData is empty (Added to eliminate callback error of data out of range)
    if viewData is None or len(viewData) == 0:
        # Return a default map or a message indicating no data
        return [html.P("No data available to display on the map.")]
    
    dff = pd.DataFrame.from_dict(viewData)

    # Select the first row by default if no row is selected
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]
    
    # Get latitude and longitude from the selected row
    lat = dff.iloc[row]['location_lat']
    long = dff.iloc[row]['location_long']
    
    # Create and return the map with a marker for the selected animal
    return [
        dl.Map(style={'width': '600px', 'height': '350px'}, center=[lat, long], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, long], children=[
                dl.Tooltip(dff.iloc[row]['breed']),
                dl.Popup([html.H1("Animal Name"), html.P(dff.iloc[row]['name'])])
            ])
        ])
    ]
# Callback to update the pie chart based on filtered data
@app.callback(
    Output('pie-chart-id', 'figure'),
    [Input('datatable-id', 'data')]
)
def update_pie_chart(data):
    # Convert the table data back into a DataFrame
    dff = pd.DataFrame(data)
    
    # Create a pie chart showing the distribution of breeds
    fig = px.pie(dff, names='breed', title='Breed Distribution of Selected Rescue Type')
    
    # Customize how the labels display on hover
    fig.update_traces(
        textinfo='percent+label',  # Display percent and breed on the slices
        hoverinfo='label+percent+value',  # Show breed, percent, and count on hover
        textposition='inside'  # Ensure the text is inside the slices
    )
    
    return fig


app.run_server(debug=True)

Dash app running on http://127.0.0.1:23817/
